In [4]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing import DisparateImpactRemover
from aif360.metrics import ClassificationMetric

In [8]:
# Load the data
train = pd.read_csv(r'../Resources/Data/baseline_train_data.csv')
test = pd.read_csv(r'../Resources/Data/baseline_test_data.csv')
valid = pd.read_csv(r'../Resources/Data/baseline_valis_data.csv')

# Define target and features
target = 'two_year_recid'

# Separate features and target for training, testing, and validation data
X_train = train.drop(columns=[target])
y_train = train[target]

X_test = test.drop(columns=[target])
y_test = test[target]

X_valid = valid.drop(columns=[target])
y_valid = valid[target]

In [10]:
# Convert to AIF360 BinaryLabelDataset format
train_aif = BinaryLabelDataset(df=pd.concat([X_train, y_train], axis=1),
                                label_names=[target],
                                protected_attribute_names=['sex'])

test_aif = BinaryLabelDataset(df=pd.concat([X_test, y_test], axis=1),
                               label_names=[target],
                               protected_attribute_names=['sex'])

In [12]:
# Apply Disparate Impact Remover
dir = DisparateImpactRemover(repair_level=1.0)
train_aif_transf = dir.fit_transform(train_aif)

# Extract the transformed features and target
X_train_transf = train_aif_transf.features
y_train_transf = train_aif_transf.labels

In [14]:
# Train the logistic regression model on the transformed data
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_transf, y_train_transf)

# Predict on the test data
y_pred_test = log_reg.predict(X_test)


C:\Users\General\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\General\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [16]:
# Define privileged and unprivileged groups
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]

In [18]:
# Convert to AIF360 BinaryLabelDataset format for test set with predictions
test_aif_pred = BinaryLabelDataset(df=pd.concat([X_test, pd.Series(y_pred_test, name=target)], axis=1),
                                   label_names=[target],
                                   protected_attribute_names=['sex'])

In [20]:
# Compute fairness metrics
metric_test_pred = ClassificationMetric(test_aif, test_aif_pred,
                                        unprivileged_groups=unprivileged_groups,
                                        privileged_groups=privileged_groups)

stat_parity_diff = metric_test_pred.statistical_parity_difference()
avg_odds_diff = metric_test_pred.average_odds_difference()
equal_opp_diff = metric_test_pred.equal_opportunity_difference()

# Display results
print("Disparate Impact Remover")
print(f"Statistical Parity Difference: {stat_parity_diff}")
print(f"Average Odds Difference: {avg_odds_diff}")
print(f"Equal Opportunity Difference: {equal_opp_diff}")
print()

Disparate Impact Remover
Statistical Parity Difference: 0.23776896724277302
Average Odds Difference: 0.22409495346851444
Equal Opportunity Difference: 0.22007930785868784

